In [3]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.Collecting opencv-python
                                              0.0/38.1 MB ? eta -:--:--
                                              0.2/38.1 MB 3.5 MB/s eta 0:00:11
                                              0.3/38.1 MB 3.5 MB/s eta 0:00:11
                                              0.5/38.1 MB 3.7 MB/s eta 0:00:11
                                              0.7/38.1 MB 3.8 MB/s eta 0:00:10
                                              0.8/38.1 MB 3.2 MB/s eta 0:00:12
                                              0.9/38.1 MB 3.1 MB/s eta 0:00:12
     -                                        1.0/38.1 MB 3.2 MB/s eta 0:00:12
     -                                        1.2/38.1 MB 3.0 MB/s eta 0:00:13
     -                                        1.2/38.1 MB 2.8 MB/s eta 0:00:14
     -                                        1.3/38.1 MB 2.6 MB/s eta 0:00:14
     -                                        1.

In [4]:
import cv2
import numpy as np
from time import sleep

In [8]:
import cv2
import numpy as np
import time

min_width_rect = 80  # Minimum width of the rectangle
min_height_rect = 80  # Minimum height of the rectangle

offset = 6  # Allowable error between pixels

count_line_pos2 = 550  # left lane ke liye
count_line_pos1 = 550  # right lane ke liye

delay = 60  # FPS

detec2 = []  # left lane ke liye
detec1 = []  # right lane ke liye

carros2 = 0  # left lane ke liye
carros1 = 0  # right lane ke liyeW

cap = cv2.VideoCapture('LEFT.mp4')
# cap1= cv2.VideoCapture('RIGHT.mp4')

algo = cv2.createBackgroundSubtractorMOG2()

# left lane
def central_handle2(p, q, r, s):
    p1 = int(r / 2)
    q1 = int(s / 2)
    cx2 = p + p1
    cy2 = q + q1
    return cx2, cy2

while True:
    ret, frame1 = cap.read()

    if not ret:
        break

    tempo = 1 / delay
    time.sleep(tempo)

    grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)  # left frame

    blur = cv2.GaussianBlur(grey, (3, 3), 5)  # size of kernel is (3,3) #grey is the image to be blurred #Kernel standard deviation along X-axis #left lane

    img_sub = algo.apply(blur)  # left lane

    dilat = cv2.dilate(img_sub, np.ones((5, 5)))  # left lane

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)  # left lane
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)  # left lane

    contours, _ = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  # left lane

    cv2.line(frame1, (25, count_line_pos2), (1200, count_line_pos2), (0, 255, 255), 3)

    for (i, c) in enumerate(contours):
        (p, q, r, s) = cv2.boundingRect(c)
        validar_contorno2 = (r >= min_width_rect) and (s >= min_height_rect)
        if not validar_contorno2:
            continue

        cv2.rectangle(frame1, (p, q), (p + r, q + s), (0, 255, 0), 2)
        cv2.putText(frame1, "Vehicle" + str(carros2), (p, q - 20), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 244, 0), 2)

        center2 = central_handle2(p, q, r, s)
        detec2.append(center2)
        cv2.circle(frame1, center2, 4, (0, 0, 255), -1)

        for (p, q) in detec2:
            if q < (count_line_pos2 + offset) and q > (count_line_pos2 - offset):
                carros2 += 1
                cv2.line(frame1, (25, count_line_pos2), (500, count_line_pos2), (0, 255, 0), 3)
                detec2.remove((p, q))
                print("Car is detected in the left lane: " + str(carros2))

    cv2.putText(frame1, "LEFT : " + str(carros2), (250, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 5)

    cv2.imshow("LEFT", frame1)
    if cv2.waitKey(1) == 13:
        break

    if carros2 <= 6:
        print("RED")
    else:
        print("GREEN")

cv2.destroyAllWindows()
cap.release()
